## Prepare data for email_out

In [17]:
from sqlalchemy.engine import create_engine
from sqlalchemy import inspect
from sqlalchemy import text
from pprint import pprint

import csv

mmCoursesMeta = {
    'buslead': {
        "buslead1x": {"title": "UQx_BUSLEAD1x",  'grade': 0.65},
        "buslead2x": {"title": "UQx_BUSLEAD2x",  'grade': 0.65},
        "buslead3x": {"title": "UQx_BUSLEAD3x",  'grade': 0.65},
        "buslead4x": {"title": "UQx_BUSLEAD4x",  'grade': 0.65},
        "buslead5x": {"title": "UQx_BUSLEAD5x",  'grade': 0.65},
    },
    'corpinn': {
        "corpinn1x": {"title": "UQx_CORPINN1x", 'grade': 0.7},
        "corpinn2x": {"title": "UQx_CORPINN2x",  'grade': 0.7},
        "corpinn3x": {"title": "UQx_CORPINN3x",  'grade': 0.7},
        "corpinn4x": {"title": "UQx_CORPINN4x",  'grade': 0.7},
        "corpinn5x": {"title": "UQx_CORPINN5x",  'grade': 0.5},
    },
    'engy': {
        "engy0x": {"title": "UQx_ENGY0x", 'grade': 0.7},
        "engy1x": {"title": "UQx_ENGY1x", 'grade': 0.7},
        "engy2x": {"title": "UQx_ENGY2x", 'grade': 0.7},
        "engy3x": {"title": "UQx_ENGY3x", 'grade': 0.7},
        "engycapx": {"title": "UQx_ENGYCAPx", 'grade': 0.7},
    },
    'lgdm': {
        "lgdm1x": {"title": "UQx_LGDM1x", 'grade': 0.65},
        "lgdm2x": {"title": "UQx_LGDM2x",  'grade': 0.65},
        "lgdm3x": {"title": "UQx_LGDM3x",  'grade': 0.65},
        "lgdm4x": {"title": "UQx_LGDM4x",  'grade': 0.65},
        "lgdm5x": {"title": "UQx_LGDM5x",  'grade': 0.65},
    },
}
mmCoursesSeries = {
    'buslead': ['buslead1x', 'buslead2x', 'buslead3x', 'buslead4x'],
    'corpinn': ['corpinn1x', 'corpinn2x', 'corpinn3x', 'corpinn4x'],
    'engy': ['engy0x', 'engy1x', 'engy2x', 'engy3x'],
    'lgdm': ['lgdm1x', 'lgdm2x', 'lgdm3x', 'lgdm4x'],
}

pointed_course_instances = {
    'lgdm1x': {
        'current_run': {'title': 'Leaders in Global Development', 'link': 'https://www.edx.org/course/leaders-in-global-development', 'instance_code': 'UQx_LGDM1x_1T2019'},
        'coming_run': {'title': 'Leaders in Global Development', 'link': 'https://www.edx.org/course/leaders-in-global-development', 'instance_code': 'UQx_LGDM1x_1T2019'},
    },
    'lgdm2x': {
        'current_run':{'title': 'The Science and Practice of Sustainable Development', 'link': 'https://www.edx.org/course/the-science-and-practice-of-sustainable-development', 'instance_code': 'UQx_LGDM2x_1T2019'},
        'coming_run': {'title': 'The Science and Practice of Sustainable Development', 'link': 'https://www.edx.org/course/the-science-and-practice-of-sustainable-development', 'instance_code': 'UQx_LGDM2x_1T2019'},
    },
    'lgdm3x': {
        'current_run':{'title': 'Adaptive Leadership in Development', 'link': 'https://www.edx.org/course/adaptive-leadership-in-development', 'instance_code': 'UQx_LGDM3x_1T2018'}, 
        'coming_run': {'title': 'Adaptive Leadership in Development', 'link': 'https://www.edx.org/course/adaptive-leadership-in-development', 'instance_code': 'UQx_LGDM3x_1T2018'},       
    },
    'lgdm4x': {
        'current_run':{'title': 'Critical Development Perspectives', 'link': 'https://www.edx.org/course/critical-development-perspectives', 'instance_code': 'UQx_LGDM4x_1T2019'},
        'coming_run': {'title': 'Critical Development Perspectives', 'link': 'https://www.edx.org/course/critical-development-perspectives', 'instance_code': 'UQx_LGDM4x_1T2019'},        
    },
    'corpinn1x': {
        'current_run':{'title': 'Design Thinking and Creativity for Innovation', 'link': 'https://www.edx.org/course/design-thinking-and-creativity-for-innovation-0', 'instance_code': 'UQx_CORPINN1x_1T2019'},
        'coming_run': {'title': 'Design Thinking and Creativity for Innovation', 'link': 'https://www.edx.org/course/design-thinking-and-creativity-for-innovation-0', 'instance_code': 'UQx_CORPINN1x_1T2019'},        
    },
    'corpinn2x': {
        'current_run':{'title': 'Idea Development: Create and Implement Innovative Ideas', 'link': 'https://www.edx.org/course/idea-development-create-and-implement-innovative-ideas', 'instance_code': 'UQx_CORPINN2x_1T2019'},
        'coming_run': {'title': 'Idea Development: Create and Implement Innovative Ideas', 'link': 'https://www.edx.org/course/idea-development-create-and-implement-innovative-ideas', 'instance_code': 'UQx_CORPINN2x_1T2019'},        
    },
    'corpinn3x': {
        'current_run':{'title': 'Innovation: From Plan to Product', 'link': 'https://www.edx.org/course/innovation-from-plan-to-product', 'instance_code': 'UQx_CORPINN3x_1T2019'},
        'coming_run': {'title': 'Innovation: From Plan to Product', 'link': 'https://www.edx.org/course/innovation-from-plan-to-product', 'instance_code': 'UQx_CORPINN3x_1T2019'},        
    },
    'corpinn4x': {
        'current_run':{'title': 'Creating and Sustaining an Innovation Culture', 'link': 'https://www.edx.org/course/creating-sustaining-innovation-culture-uqx-corpinn4x', 'instance_code': 'UQx_CORPINN3x_1T2019'},
        'coming_run': {'title': 'Creating and Sustaining an Innovation Culture', 'link': 'https://www.edx.org/course/creating-sustaining-innovation-culture-uqx-corpinn4x', 'instance_code': 'UQx_CORPINN3x_1T2019'},        
    },
    'buslead1x': {
        'current_run':{'title': 'Becoming an Effective Leader', 'link': 'https://www.edx.org/course/becoming-an-effective-leader-0', 'instance_code': 'UQx_BUSLEAD1x_1T2019'},
        'coming_run': {'title': 'Becoming an Effective Leader', 'link': 'https://www.edx.org/course/becoming-an-effective-leader-0', 'instance_code': 'UQx_BUSLEAD1x_1T2019'},        
    },
    'buslead2x': {
        'current_run':{'title': 'Leading the Organization', 'link': 'https://www.edx.org/course/leading-the-organization-0', 'instance_code': 'UQx_BUSLEAD2x_1T2019'},
        'coming_run': {'title': 'Leading the Organization', 'link': 'https://www.edx.org/course/leading-the-organization-0', 'instance_code': 'UQx_BUSLEAD2x_1T2019'},        
    },
    'buslead3x': {
        'current_run':{'title': 'Leading High-Performing Teams', 'link': 'https://www.edx.org/course/leading-high-performing-teams', 'instance_code': 'UQx_BUSLEAD3x_1T2019'},
        'coming_run': {'title': 'Leading High-Performing Teams', 'link': 'https://www.edx.org/course/leading-high-performing-teams', 'instance_code': 'UQx_BUSLEAD3x_1T2019'},        
    },
    'buslead4x': {
        'current_run':{'title': 'Leading in a Complex Environment', 'link': 'https://www.edx.org/course/leading-in-a-complex-environment', 'instance_code': 'UQx_BUSLEAD4x_1T2019'},
        'coming_run': {'title': 'Leading in a Complex Environment', 'link': 'https://www.edx.org/course/leading-in-a-complex-environment', 'instance_code': 'UQx_BUSLEAD4x_1T2019'},        
    },
    'engy0x': {
        'current_run':{'title': 'Energy Principles and Renewable Energy', 'link': 'https://www.edx.org/course/energy-principles-and-renewable-energy-2', 'instance_code': 'UQx_ENGY0x_1T2019'},
        'coming_run': {'title': 'Energy Principles and Renewable Energy', 'link': 'https://www.edx.org/course/energy-principles-and-renewable-energy-2', 'instance_code': 'UQx_ENGY0x_1T2019a'},        
    },
    'engy1x': {
        'current_run':{'title': 'Climate Science and Policy', 'link': 'https://www.edx.org/course/climate-science-and-policy-2', 'instance_code': 'UQx_ENGY0x_1T2019'},
        'coming_run': {'title': 'Climate Science and Policy', 'link': 'https://www.edx.org/course/climate-science-and-policy-2', 'instance_code': 'UQx_ENGY1x_1T2019a'},        
    },
    'engy2x': {
        'current_run':{'title': 'Energy and Development', 'link': 'https://www.edx.org/course/energy-and-development-1', 'instance_code': 'UQx_ENGY2x_1T2019'},
        'coming_run': {'title': 'Energy and Development', 'link': 'https://www.edx.org/course/energy-and-development-1', 'instance_code': 'UQx_ENGY2x_2T2019'},        
    },
    'engy3x': {
        'current_run':{'title': 'Low Emission Technologies and Supply Systems', 'link': 'https://www.edx.org/course/low-emission-technologies-and-supply-systems-0', 'instance_code': 'course-v1:UQx_ENGY3x_1T2019'},
        'coming_run': {'title': 'Low Emission Technologies and Supply Systems', 'link': 'https://www.edx.org/course/low-emission-technologies-and-supply-systems-0', 'instance_code': 'course-v1:UQx_ENGY3x_2T2019'},        
    },    
}  

def update_course_instancedetails_auditpassed():
    print('update_course_instancedetails_auditpassed')
    ci_query = 'SELECT * FROM course_instancedetails'
    result = sqlite_connection.execute(ci_query)
    for row in result:
        update_id = int(row[0])
        course = row[6]
        pass_line = micromasters_courses[course]['grade']
        #print('pass_line', pass_line)
        grade = row[2]
        enrollment_track = row[3]
        update_ci = "UPDATE course_instancedetails SET audit_passed='%s' WHERE id=%d; " %('N', update_id)
        if (enrollment_track == 'audit') and (grade >= pass_line):
            update_ci = "UPDATE course_instancedetails SET audit_passed='%s' WHERE id=%d; " %('Y', update_id)
        sqlite_connection.execute(update_ci)
        
        
def micromasters_courses_add_last_run():
    for course in micromasters_courses:
        query = "SELECT course_instance FROM course_instancedetails WHERE course='%s' ORDER BY year DESC, term DESC;" % (course)
        query_result = sqlite_connection.execute(query).fetchone()
        if query_result is not None:
            micromasters_courses[course]['last_run'] = query_result[0]
    

def find_auditpassed(course):
    trigger_run = pointed_course_instances[course]['current_run']['instance_code']
    trigger_course_title = pointed_course_instances[course]['current_run']['title']
    trigger_course_link = pointed_course_instances[course]['current_run']['link']
    
    query = "SELECT c.learner_id, c.course_instance, c.audit_passed, l.name, l.email FROM course_instancedetails c, learners l WHERE c.learner_id = l.student_id AND c.course_instance = '%s' AND c.audit_passed = 'Y'" % (trigger_run)
    query_result = sqlite_connection.execute(query)
    for record in query_result:
        result = {}
        result['email'] = record['email']
        result['student_name'] = record['name']
        result['student_id'] = record['learner_id']
        result['trigger_course_verified'] = 0
        result['trigger_course_title'] = trigger_course_title
        result['trigger_course_link'] = trigger_course_link
        filtered_students.append(result)
    
def find_certificate_eligible(course):
    trigger_run = pointed_course_instances[course]['current_run']['instance_code']
    trigger_course_title = pointed_course_instances[course]['current_run']['title']
    trigger_course_link = pointed_course_instances[course]['current_run']['link']

    query = "SELECT c.learner_id, c.course_instance, c.audit_passed, l.name, l.email FROM course_instancedetails c, learners l WHERE c.learner_id = l.student_id AND c.course_instance = '%s' AND c.certificate_eligible = 'Y'" % (trigger_run)
    query_result = sqlite_connection.execute(query)
    for record in query_result:
        result = {}
        result['email'] = record['email']
        result['student_name'] = record['name']
        result['student_id'] = record['learner_id']
        result['trigger_course_verified'] = 1
        result['trigger_course_title'] = trigger_course_title
        result['trigger_course_link'] = trigger_course_link
        filtered_students.append(result)


def find_auditpassed_next_course(course, record):
    #print('find_next_course', course, record)
    not_certificated = []
    
    if not (record[5] == 'verified' and record[6] == 'Y'):
        not_certificated.append(micromasters_course_series[0])
    if not (record[7] == 'verified' and record[8] == 'Y'):
        not_certificated.append(micromasters_course_series[1])
    if not (record[9] == 'verified' and record[10] == 'Y'):
        not_certificated.append(micromasters_course_series[2])
    if not (record[11] == 'verified' and record[12] == 'Y'):
        not_certificated.append(micromasters_course_series[3])
    #print(not_certificated)
    if course in not_certificated:
        not_certificated.remove(course)
    return not_certificated[0]

def find_certificate_eligible_next_course(record):
    not_certificated = []
    
    #print('record', record)
    
    if not (record[5] == 'verified' and record[6] == 'Y'):
        not_certificated.append(micromasters_course_series[0])
    if not (record[7] == 'verified' and record[8] == 'Y'):
        not_certificated.append(micromasters_course_series[1])
    if not (record[9] == 'verified' and record[10] == 'Y'):
        not_certificated.append(micromasters_course_series[2])
    if not (record[11] == 'verified' and record[12] == 'Y'):
        not_certificated.append(micromasters_course_series[3])
    #print(not_certificated)
    return not_certificated[0]
    
    


mmCourse = 'engy'
course = 'engy0x'
table_name = mmCourse + '_coursecompletion'  
micromasters_courses = mmCoursesMeta[mmCourse]
micromasters_course_series = mmCoursesSeries[mmCourse]

filtered_students = []



sqlite_engine = create_engine('sqlite:////Users/saisun/ProjectCode/uqx_tools/micromastersreports/databases/' + mmCourse + '_verified.db')
sqlite_connection = sqlite_engine.connect()
  
    
# Prepare 1, update the column 'audit_passed' in the table {course_instancedetails}        
update_course_instancedetails_auditpassed()   

# Prepare 2, get the last run. Maybe not needed now, as we have the variable 'pointed_course_instances'
# micromasters_courses_add_last_run()

find_auditpassed(course)
find_certificate_eligible(course)

for item in filtered_students:
    query = "SELECT * FROM %s WHERE email = '%s'" % (table_name, item['email'])
    query_result = sqlite_connection.execute(query)
    for record in query_result:
        item['no_certificate_eligible'] = record['no_certificate_eligible']
        if (item['trigger_course_verified'] == 0) and (item['no_certificate_eligible'] < 3):
            next_course = find_auditpassed_next_course(course, record)
            item['next_course_title'] = pointed_course_instances[next_course]['coming_run']['title']
            item['next_course_link'] = pointed_course_instances[next_course]['coming_run']['link']
        if (item['trigger_course_verified'] == 1) and (item['no_certificate_eligible'] < 4):
            next_course = find_certificate_eligible_next_course(record)
            item['next_course_title'] = pointed_course_instances[next_course]['coming_run']['title']
            item['next_course_link'] = pointed_course_instances[next_course]['coming_run']['link']
        # if the student has achieved all four course certified, we care not going to email him
        if item['no_certificate_eligible'] == 4:
            #print('all certified')
            filtered_students.remove(item)
            
print(filtered_students)
    

print('The End.')



update_course_instancedetails_auditpassed
[{'email': 'sushmitsdutta@gmail.com', 'student_name': 'Sushmit Dutta', 'student_id': 7842549, 'trigger_course_verified': 1, 'trigger_course_title': 'Energy Principles and Renewable Energy', 'trigger_course_link': 'https://www.edx.org/course/energy-principles-and-renewable-energy-2', 'no_certificate_eligible': 1, 'next_course_title': 'Climate Science and Policy', 'next_course_link': 'https://www.edx.org/course/climate-science-and-policy-2'}, {'email': 'Alex.Farah@shell.com', 'student_name': 'Farah', 'student_id': 20080412, 'trigger_course_verified': 1, 'trigger_course_title': 'Energy Principles and Renewable Energy', 'trigger_course_link': 'https://www.edx.org/course/energy-principles-and-renewable-energy-2', 'no_certificate_eligible': 1, 'next_course_title': 'Climate Science and Policy', 'next_course_link': 'https://www.edx.org/course/climate-science-and-policy-2'}]
The End.


## Dynamic email out

In [18]:
from keys_list import aws_access_key_id, aws_secret_access_key, unsubscribe_secret
from email.utils import COMMASPACE
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import boto.ses
import jinja2
import hashlib
#from pprint import pprint

from_address = '"UQx Updates" <business@uqx.uq.edu.au>'
subject = 'Congratulations - you are now one step closer to the MicroMasters credential!'

templateLoader = jinja2.FileSystemLoader(searchpath="./dynamic_email_templates")
templateEnv = jinja2.Environment(loader=templateLoader)

template_files = {
    'buslead': 'buslead_dynamic',
    'corpinn': 'corpinn_dynamic',
    'engy': 'engy_dynamic',
    'lgdm': 'lgdm_dynamic'  
}

text_file = template_files[mmCourse] + '.txt'
html_file = template_files[mmCourse] + '.html'

#print(text_file, html_file)

test_user = [{'email': 'uqssun@uq.edu.au', 'student_name': 'Sai Sun', 'student_id': 10553000, 'trigger_course_verified': 0, 'trigger_course_title': pointed_course_instances[course]['current_run']['title'], 'trigger_course_link': pointed_course_instances[course]['current_run']['link'], 'no_certificate_eligible': 1, 'next_course_title': pointed_course_instances[next_course]['coming_run']['title'], 'next_course_link': pointed_course_instances[next_course]['coming_run']['link']}]
email_list = filtered_students + test_user
print(email_list)

def render(template_file, context_dict):
    template = templateEnv.get_template(template_file)
    output = template.render(item=context_dict)
    #print(output)
    return output

def get_md5_unsubsecret(email_opt_out):
    m = hashlib.md5()
    m.update((unsubscribe_secret + email_opt_out).encode('utf-8'))
    return m.hexdigest()

def make_unsubscribe_link(email_opt_out):
    email_opt_out = email_opt_out.lower()

    validation_hash = get_md5_unsubsecret(email_opt_out)

    link = 'https://tools.ceit.uq.edu.au/uqxmailouts/unsubscribe/index.php?email=' + email_opt_out + '&validation_hash=' + validation_hash
    #print('link', link)
    return link

class SESMessage(object):

    def __init__(self, source, to_addresses, subject, body_html, body_text, **kw):

        self.ses = boto.ses.connect_to_region('us-west-2', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

        self._source = source
        self._to_addresses = to_addresses
        self._cc_addresses = None
        self._bcc_addresses = None

        self.subject = subject
        self.text = body_text
        self.html = body_html
        self.attachments = []

    def send(self):
        #print('sending')        
        if not self.ses:
            raise Exception('No connection found')

        if (self.text and not self.html and not self.attachments) or\
           (self.html and not self.text and not self.attachments):
            return self.ses.send_email(self._source, self.subject,
                self.text or self.html,
                self._to_addresses, self._cc_addresses,
                self._bcc_addresses,
                format='text' if self.text else 'html')
        else:
            if not self.attachments:
                message = MIMEMultipart('alternative')
                message['subject'] = self.subject
                message['from'] = self._source
                if isinstance(self._to_addresses, (list, tuple)):
                    message['To'] = COMMASPACE.join(self._to_addresses)
                else:
                    message['To'] = self._to_addresses

                message.attach(MIMEText(self.text, 'plain'))
                message.attach(MIMEText(self.html, 'html'))
                #print(message)
            else:
                raise NotImplementedError
            

        return self.ses.send_raw_email(message.as_string(), source=self._source, destinations=self._to_addresses)

#for item in email_list:         
for item in test_user:
    item['unsubscribe_link'] = make_unsubscribe_link(item['email'])
    #print(item)
    body_text = render(text_file, item)
    body_html = render(html_file, item)
    sesMessage = SESMessage(from_address, item['email'], subject, body_html, body_text)
    sesMessage.send()





[{'email': 'sushmitsdutta@gmail.com', 'student_name': 'Sushmit Dutta', 'student_id': 7842549, 'trigger_course_verified': 1, 'trigger_course_title': 'Energy Principles and Renewable Energy', 'trigger_course_link': 'https://www.edx.org/course/energy-principles-and-renewable-energy-2', 'no_certificate_eligible': 1, 'next_course_title': 'Climate Science and Policy', 'next_course_link': 'https://www.edx.org/course/climate-science-and-policy-2'}, {'email': 'Alex.Farah@shell.com', 'student_name': 'Farah', 'student_id': 20080412, 'trigger_course_verified': 1, 'trigger_course_title': 'Energy Principles and Renewable Energy', 'trigger_course_link': 'https://www.edx.org/course/energy-principles-and-renewable-energy-2', 'no_certificate_eligible': 1, 'next_course_title': 'Climate Science and Policy', 'next_course_link': 'https://www.edx.org/course/climate-science-and-policy-2'}, {'email': 'uqssun@uq.edu.au', 'student_name': 'Sai Sun', 'student_id': 10553000, 'trigger_course_verified': 0, 'trigger_c